In [2]:
!pip3 install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 4.5 MB/s eta 0:00:00 MB/s eta 0:00:01


In [1]:
import eikon as ek
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")
import configparser as cp
#https://github.com/Refinitiv-API-Samples/Article.EikonAPI.Python.NewsSentimentAnalysis

In [2]:
cfg = cp.ConfigParser()
cfg.read('../refinitiv.cfg')
ek.set_app_key(cfg['eikon']['app_id'])

In [3]:
from datetime import datetime, timedelta
end = datetime.today()
start = end - timedelta(days=(365.2*4))
print(start, end)

2019-04-14 00:26:44.962372 2023-04-13 19:38:44.962372


In [19]:
df = ek.get_news_headlines('R:SAP.N AND Language:LEN', date_from='2023-04-12T12:05:22', dateTo='2023-03-14T13:31:00', count=100)
df.head()

TypeError: get_news_headlines() got an unexpected keyword argument 'dateTo'

In [5]:
df['Polarity'] = np.nan
df['Subjectivity'] = np.nan
df['Score'] = np.nan

In [6]:
for idx, storyId in enumerate(df['storyId'].values):  #for each row in our df dataframe
    newsText = ek.get_news_story(storyId) #get the news story
    if newsText:
        soup = BeautifulSoup(newsText,"lxml") #create a BeautifulSoup object from our HTML news article
        sentA = TextBlob(soup.get_text()) #pass the text only article to TextBlob to anaylse
        df['Polarity'].iloc[idx] = sentA.sentiment.polarity #write sentiment polarity back to df
        df['Subjectivity'].iloc[idx] = sentA.sentiment.subjectivity #write sentiment subjectivity score back to df
        if sentA.sentiment.polarity >= 0.05: # attribute bucket to sentiment polartiy
            score = 'positive'
        elif  -.05 < sentA.sentiment.polarity < 0.05:
            score = 'neutral'
        else:
            score = 'negative'
        df['Score'].iloc[idx] = score #write score back to df
df.head()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score
2023-04-13 05:22:36.026,2023-04-13 05:22:36.026000+00:00,Construction Of Pit In Sap.,urn:newsml:newswire.refinitiv.com:20230413:nNR...,NS:ECLTND,0.0,0.0,neutral
2023-04-12 12:10:42.984,2023-04-12 12:10:43.247000+00:00,(EN) SAP SE Annual General Meeting (73138162),urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral
2023-04-12 12:08:49.558,2023-04-12 12:09:17.591000+00:00,(EN) SAP SE Notice of AGM (73138134),urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral
2023-04-12 12:08:23.372,2023-04-12 12:08:27.124000+00:00,(EN) SAP SE Code of Ethics (73138084),urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral
2023-04-12 12:05:20.164,2023-04-12 12:05:22.542000+00:00,(EN) SAP SE Governance Principles / Guidelines...,urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral


In [7]:
start = df['versionCreated'].min().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
end = df['versionCreated'].max().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y/%m/%d')
Minute = ek.get_timeseries(["SAP.N"], start_date=start, interval="minute")
Minute

SAP.N,HIGH,LOW,OPEN,CLOSE,COUNT,VOLUME
Date,,,,,,
2023-03-14 13:31:00,118.22,118.02,118.07,118.22,6,22733
2023-03-14 13:32:00,118.17,118.04,118.17,118.04,9,500
2023-03-14 13:33:00,118.07,118.03,118.07,118.03,2,103
2023-03-14 13:34:00,117.89,117.89,117.89,117.89,1,2
2023-03-14 13:35:00,117.93,117.79,117.93,117.79,4,225
...,...,...,...,...,...,...
2023-04-13 20:00:00,128.09,128.01,128.09,128.05,97,18716
2023-04-13 20:01:00,128.05,128.05,128.05,128.05,2,40066
2023-04-13 20:11:00,128.05,128.05,128.05,128.05,1,<NA>


In [8]:
df['twoM'] = np.nan
df['fiveM'] = np.nan
df['tenM'] = np.nan
df['thirtyM'] = np.nan
df.head(2)

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-04-13 05:22:36.026,2023-04-13 05:22:36.026000+00:00,Construction Of Pit In Sap.,urn:newsml:newswire.refinitiv.com:20230413:nNR...,NS:ECLTND,0.0,0.0,neutral,NaN,NaN,NaN,NaN
2023-04-12 12:10:42.984,2023-04-12 12:10:43.247000+00:00,(EN) SAP SE Annual General Meeting (73138162),urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral,NaN,NaN,NaN,NaN


In [9]:
df.tail()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-03-14 14:08:07.532,2023-03-14 14:08:07+00:00,SHAREHOLDER NOTICE: Halper Sadeh LLC Investiga...,urn:newsml:newsroom.refinitiv.com:20230314:nVM...,NS:CMNW,0.091387,0.293697,positive,NaN,NaN,NaN,NaN
2023-03-14 14:08:04.985,2023-03-14 14:08:04+00:00,SHAREHOLDER NOTICE: Halper Sadeh LLC Investiga...,urn:newsml:newsroom.refinitiv.com:20230314:nVM...,NS:CMNW,0.091387,0.293697,positive,NaN,NaN,NaN,NaN
2023-03-14 13:00:00.449,2023-03-14 13:00:00.449000+00:00,Qualtrics Partners With the DAISY Foundation™ ...,urn:newsml:newswire.refinitiv.com:20230314:nBw...,NS:BSW,0.282062,0.506132,positive,NaN,NaN,NaN,NaN
2023-03-14 12:09:41.854,2023-03-14 12:09:41.854000+00:00,NTT DATA Business Solutions AG Receives SAP® M...,urn:newsml:newswire.refinitiv.com:20230314:nPr...,NS:PRN,0.228529,0.418941,positive,NaN,NaN,NaN,NaN
2023-03-14 09:41:47.000,2023-03-14 09:41:47+00:00,Deals of the day-Mergers and acquisitions,urn:newsml:newswire.refinitiv.com:20230314:nL4...,NS:RTRS,0.076471,0.320588,positive,NaN,NaN,NaN,NaN


In [10]:
for idx, newsDate in enumerate(df['versionCreated'].values):
    sTime = df['versionCreated'][idx]
    sTime = sTime.replace(second=0,microsecond=0)
    try:
        t0 = Minute.iloc[Minute.index.get_loc(sTime),2]
        df['twoM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + datetime.timedelta(minutes=2))),3]/(t0)-1)*100)
        df['fiveM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + datetime.timedelta(minutes=5))),3]/(t0)-1)*100)
        df['tenM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + datetime.timedelta(minutes=10))),3]/(t0)-1)*100) 
        df['thirtyM'][idx] = ((Minute.iloc[Minute.index.get_loc((sTime + datetime.timedelta(minutes=30))),3]/(t0)-1)*100)
    except:
        pass
df.head()

,versionCreated,text,storyId,sourceCode,Polarity,Subjectivity,Score,twoM,fiveM,tenM,thirtyM
2023-04-13 05:22:36.026,2023-04-13 05:22:36.026000+00:00,Construction Of Pit In Sap.,urn:newsml:newswire.refinitiv.com:20230413:nNR...,NS:ECLTND,0.0,0.0,neutral,NaN,NaN,NaN,NaN
2023-04-12 12:10:42.984,2023-04-12 12:10:43.247000+00:00,(EN) SAP SE Annual General Meeting (73138162),urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral,NaN,NaN,NaN,NaN
2023-04-12 12:08:49.558,2023-04-12 12:09:17.591000+00:00,(EN) SAP SE Notice of AGM (73138134),urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral,NaN,NaN,NaN,NaN
2023-04-12 12:08:23.372,2023-04-12 12:08:27.124000+00:00,(EN) SAP SE Code of Ethics (73138084),urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral,NaN,NaN,NaN,NaN
2023-04-12 12:05:20.164,2023-04-12 12:05:22.542000+00:00,(EN) SAP SE Governance Principles / Guidelines...,urn:newsml:newswire.refinitiv.com:20230412:nGL...,NS:GLFILE,0.0,0.0,neutral,NaN,NaN,NaN,NaN


In [11]:
grouped = df.groupby(['Score']).mean()
grouped

,Polarity,Subjectivity,twoM,fiveM,tenM,thirtyM
Score,,,,,,
negative,-0.066667,0.248333,NaN,NaN,NaN,NaN
neutral,0.010080,0.140081,NaN,NaN,NaN,NaN
positive,0.152216,0.402025,NaN,NaN,NaN,NaN
